In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from keras.wrappers import KerasClassifier

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
ds_train = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/train',
    labels='inferred',
    label_mode='binary',
    image_size=[64,64],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
)

Found 100000 files belonging to 2 classes.


In [4]:
ds_test = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/test',
    labels='inferred',
    label_mode='binary',
    image_size=[64,64],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
)

Found 20000 files belonging to 2 classes.


In [5]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [6]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [7]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [8]:

def create_model(learning_rate=0.001, dropout_rate=0.0):
    model = Sequential([
        layers.InputLayer(input_shape=[64, 64, 3]),
        preprocessing.RandomFlip(mode='horizontal'),
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=2, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=2, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Conv2D(filters=2, kernel_size=3, activation='relu', padding='same'),
        layers.Conv2D(filters=2, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPool2D(),
        layers.BatchNormalization(renorm=True),
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dropout(dropout_rate),  
        layers.Dense(1, activation='sigmoid'),
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=0.01)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['binary_accuracy'],
    )
    return model


In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [10]:
keras_model = KerasClassifier(
    build_fn=create_model,
    epochs=50,
    verbose=0,
    callbacks=[early_stopping]  # Pass the early stopping callback here
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18752\1902238572.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasClassifier(


In [11]:
param_grid = {
    'learning_rate': [0.001,  0.01,  0.1],
    'dropout_rate': [0.0,  0.1,  0.2],
}


In [12]:
# Convert TensorFlow datasets to numpy arrays
X_train = []
y_train = []
for images, labels in ds_train:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, y_train)

C:\Users\Admin\miniconda3_2\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Admin\miniconda3_2\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "C:\Users\Admin\miniconda3_2\lib\site-packages\sklearn\metrics\_scorer.py", line 415, in __call__
    return estimator.score(*args, **kwargs)
  File "C:\Users\Admin\miniconda3_2\lib\site-packages\keras\wrappers\scikit_learn.py", line 335, in score
    raise ValueError(
ValueError: The model is not configured to compute accuracy. You should pass `metrics=["accuracy"]` to the `model.compile()` method.

  warnings.warn(


Exception ignored in: <function _EagerDefinedFunctionDeleter.__del__ at 0x0000028D17AA1480>
Traceback (most recent call last):
  File "C:\Users\Admin\miniconda3_2\lib\site-packages\tensorflow\python\eager\function.py", line 305, in __del__
    context.remove_function(self.name)
  File "C:\Users\Admin\miniconda3_2\lib\site-packages\tensorflow\python\eager\context.py", line 2740, in remove_function
    context().remove_function(name)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
    callbacks=[early_stopping],
)

# Plot learning curves
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();
